# TFX Keras Component Tutorial

***A Component-by-Component Introduction to TensorFlow Extended (TFX)***

This tutorial will interactively walk through each built-in component of TensorFlow Extended (TFX).

It covers every step in an end-to-end machine learning pipeline, from data ingestion to pushing a model to serving.

When you're done, the contents of this notebook can be automatically exported as TFX pipeline source code, which you can orchestrate with Apache Airflow and Apache Beam.

Note: This notebook demonstrates the use of native Keras models in TFX pipelines. **TFX only supports the TensorFlow 2 version of Keras**.

Note: This notebook and its associated APIs are **experimental** and are in active development.  Major changes in functionality, behavior, and presentation are expected.

## Background
This notebook demonstrates how to use TFX in a Jupyter environment.  Here, we walk through the Bike sharing example in an interactive notebook.

Working in an interactive notebook is a useful way to become familiar with the structure of a TFX pipeline.  It's also useful when doing development of your own pipelines as a lightweight development environment, but you should be aware that there are differences in the way interactive notebooks are orchestrated, and how they access metadata artifacts.

### Orchestration

In a production deployment of TFX, you will use an orchestrator such as Apache Airflow, Kubeflow Pipelines, or Apache Beam to orchestrate a pre-defined pipeline graph of TFX components.  In an interactive notebook, the notebook itself is the orchestrator, running each TFX component as you execute the notebook cells.

### Metadata

In a production deployment of TFX, you will access metadata through the ML Metadata (MLMD) API.  MLMD stores metadata properties in a database such as MySQL or SQLite, and stores the metadata payloads in a persistent store such as on your filesystem.  In an interactive notebook, both properties and payloads are stored in an ephemeral SQLite database in the `/tmp` directory on the Jupyter notebook or Colab server.

## Setup
First, we install and import the necessary packages, set up paths, and download data.

In [1]:
#!pip install tensorflow==2.2.0 tfx==0.22.0

### Import packages
We import necessary packages, including standard TFX component classes.

In [2]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input

# to create a custom metadata store connection
from ml_metadata import metadata_store
from ml_metadata.proto import metadata_store_pb2

from google.cloud import storage
client = storage.Client()

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

/opt/conda/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '


Let's check the library versions.

In [3]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.2.0
TFX version: 0.22.0


### Set up pipeline paths

In [4]:
# Data file path source
_data_filepath = "gs://bike-sharing-data/"

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), 'serving_model/bikesharing')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

### Create GS bucket and cloud sql my sql instance

* Create a GS bucket to set as pipeline root
* Create a Cloud sql my sql instance
    * Create a Cloud sql my sql instance with password and public ip address
    * Set the ip address of the VM of jupyter notebook to static
    * Autherise the VM's ip address to access cloud my sql instance
    

### Create the InteractiveContext
Last, we create an InteractiveContext, which will allow us to run TFX components interactively in this notebook.

In [5]:
_bucket = 'bike-sharing-pipeline-metadata'
_pipeline_root_gs = "gs://" + _bucket + "/tfx-pipeline"

In [6]:
# test cloud sql mysql database
connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.mysql.host = '34.76.218.49'
connection_config.mysql.port = 3306
connection_config.mysql.database = 'mlmetadata'
connection_config.mysql.user = 'root'
connection_config.mysql.password = 'hellomlmd'
store = metadata_store.MetadataStore(connection_config)

INFO:absl:MetadataStore with DB connection initialized


In [7]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
#context = InteractiveContext(pipeline_root = _pipeline_root)
context = InteractiveContext(pipeline_root = _pipeline_root_gs, metadata_connection_config = connection_config)

## Run TFX components interactively
In the cells that follow, we create TFX components one-by-one, run each of them, and visualize their output artifacts.

### ExampleGen

The `ExampleGen` component is usually at the start of a TFX pipeline. It will:

1.   Split data into training and evaluation sets (by default, 2/3 training + 1/3 eval)
2.   Convert data into the `tf.Example` format
3.   Copy data into the `_tfx_root` directory for other components to access

`ExampleGen` takes as input the path to your data source. In our case, this is the `_data_root` path that contains the downloaded CSV.

Note: In this notebook, we can instantiate components one-by-one and run them with `InteractiveContext.run()`. By contrast, in a production setting, we would specify all the components upfront in a `Pipeline` to pass to the orchestrator (see the "Export to Pipeline" section).

In [8]:
example_gen = CsvExampleGen(input=external_input(_data_filepath))
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span = None
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Using 1 process(es) for Beam pipeline execution.


INFO:absl:Processing input csv data gs://bike-sharing-data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 9
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: gs://bike-sharing-pipeline-metadata/tfx-pipeline/CsvExampleGen/examples/9, id: 10)]
        ))

Let's examine the output artifacts of `ExampleGen`. This component produces two artifacts, training examples and evaluation examples:

Note: The `%%skip_for_export` cell magic will omit the contents of this cell in the exported pipeline file (see the "Export to pipeline" section). This is useful for notebook-specific code that you don't want to run in an orchestrated pipeline.

In [9]:
%%skip_for_export

artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] gs://bike-sharing-pipeline-metadata/tfx-pipeline/CsvExampleGen/examples/9
This cell will be skipped during export to pipeline.


We can also take a look at the first three training examples:

In [11]:
%%skip_for_export

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = []
for blob in client.list_blobs(_bucket, prefix=train_uri.replace("gs://" + _bucket + "/", "")):
    tfrecord_filenames.append("gs://" + _bucket + "/" + str(blob.name))
tfrecord_filenames = tfrecord_filenames[1:]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "atemp"
    value {
      float_list {
        value: 13.635000228881836
      }
    }
  }
  feature {
    key: "casual"
    value {
      int64_list {
        value: 8
      }
    }
  }
  feature {
    key: "count"
    value {
      int64_list {
        value: 40
      }
    }
  }
  feature {
    key: "datetime"
    value {
      bytes_list {
        value: "2011-01-01 01:00:00"
      }
    }
  }
  feature {
    key: "daytype"
    value {
      bytes_list {
        value: "weekend"
      }
    }
  }
  feature {
    key: "humidity"
    value {
      int64_list {
        value: 80
      }
    }
  }
  feature {
    key: "registered"
    value {
      int64_list {
        value: 32
      }
    }
  }
  feature {
    key: "season"
    value {
      bytes_list {
        value: "spring"
      }
    }
  }
  feature {
    key: "temp"
    value {
      float_list {
        value: 9.020000457763672
      }
    }
  }
  feature {
    key: "weather"
    value {
      

Now that `ExampleGen` has finished ingesting the data, the next step is data analysis.

### StatisticsGen
The `StatisticsGen` component computes statistics over your dataset for data analysis, as well as for use in downstream components. It uses the [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) library.

`StatisticsGen` takes as input the dataset we just ingested using `ExampleGen`.

In [12]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Generating statistics for split train
INFO:absl:Statistics for split train written to gs://bike-sharing-pipeline-metadata/tfx-pipeline/StatisticsGen/statistics/10/train.
INFO:absl:Generating statistics for split eval
INFO:absl:Statistics for split eval written to gs://bike-sharing-pipeline-metadata/tfx-pipeline/StatisticsGen/statistics/10/eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 10
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: gs://bike-sharing-pipeline-metadata/tfx-pipeline/StatisticsGen/statistics/10, id: 11)]
        ))

After `StatisticsGen` finishes running, we can visualize the outputted statistics. Try playing with the different plots!

In [13]:
%%skip_for_export

context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


This cell will be skipped during export to pipeline.


### SchemaGen

The `SchemaGen` component generates a schema based on your data statistics. (A schema defines the expected bounds, types, and properties of the features in your dataset.) It also uses the [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) library.

`SchemaGen` will take as input the statistics that we generated with `StatisticsGen`, looking at the training split by default.

In [14]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Infering schema from statistics.
INFO:absl:Schema written to gs://bike-sharing-pipeline-metadata/tfx-pipeline/SchemaGen/schema/11/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 11
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: gs://bike-sharing-pipeline-metadata/tfx-pipeline/SchemaGen/schema/11, id: 12)]
        ))

After `SchemaGen` finishes running, we can visualize the generated schema as a table.

In [15]:
%%skip_for_export

context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'datetime',BYTES,required,single,-
'daytype',STRING,required,single,'daytype'
'season',STRING,required,single,'season'
'weather',STRING,required,single,'weather'
'atemp',FLOAT,required,single,-
'casual',INT,required,single,-
'count',INT,required,single,-
'humidity',INT,required,single,-
'registered',INT,required,single,-


,Values
Domain,
'daytype',"'holiday', 'weekend', 'workingday'"
'season',"'fall', 'spring', 'summer', 'winter'"
'weather',"'clear', 'cloudy', 'heavy rain', 'light rain'"


This cell will be skipped during export to pipeline.


Each feature in your dataset shows up as a row in the schema table, alongside its properties. The schema also captures all the values that a categorical feature takes on, denoted as its domain.

To learn more about schemas, see [the SchemaGen documentation](https://www.tensorflow.org/tfx/guide/schemagen).

### ExampleValidator
The `ExampleValidator` component detects anomalies in your data, based on the expectations defined by the schema. It also uses the [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) library.

`ExampleValidator` will take as input the statistics from `StatisticsGen`, and the schema from `SchemaGen`.

By default, it compares the statistics from the evaluation split to the schema from the training split.

In [16]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics.
INFO:absl:Validation complete. Anomalies written to gs://bike-sharing-pipeline-metadata/tfx-pipeline/ExampleValidator/anomalies/12.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 12
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(type_name: ExampleAnomalies, uri: gs://bike-sharing-pipeline-metadata/tfx-pipeline/ExampleValidator/anomalies/12, id: 13)]
        ))

After `ExampleValidator` finishes running, we can visualize the anomalies as a table.

In [17]:
%%skip_for_export

context.show(example_validator.outputs['anomalies'])

This cell will be skipped during export to pipeline.


In the anomalies table, we can see that we have no anomalies.

### Transform
The `Transform` component performs feature engineering for both training and serving. It uses the [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started) library.

`Transform` will take as input the data from `ExampleGen`, the schema from `SchemaGen`, as well as a module that contains user-defined Transform code.

Let's see an example of user-defined Transform code below (for an introduction to the TensorFlow Transform APIs, [see the tutorial](https://www.tensorflow.org/tfx/tutorials/transform/simple)). First, we define a few constants for feature engineering:

Note: The `%%writefile` cell magic will save the contents of the cell as a `.py` file on disk. This allows the `Transform` component to load your code as a module.



In [18]:
_bikesharing_constants_module_file = 'bikesharing_constants.py'

In [19]:
%%skip_for_export
%%writefile {_bikesharing_constants_module_file}

# Features, labels, and key columns
NUMERIC_FEATURE_KEYS=["temp", "atemp", "humidity", "windspeed"] 
CATEGORICAL_FEATURE_KEYS=["season", "weather", "daytype"] 
KEY_COLUMN = "datetime"
LABEL_COLUMN = "count"

# Modeling parameters
HIDDEN_UNITS_1 = 16
HIDDEN_UNITS_2 = 16
HIDDEN_UNITS_3 = 16
BATCH_SIZE = 128
NUM_EPOCHS = 1
LEARNING_RATE = 0.005

def transformed_name(key):
    return key 

Overwriting bikesharing_constants.py
This cell will be skipped during export to pipeline.


Next, we write a `preprocessing_fn` that takes in raw data as input, and returns transformed features that our model can train on:

In [20]:
_bikesharing_transform_module_file = 'bikesharing_transform.py'

In [21]:
%%skip_for_export
%%writefile {_bikesharing_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import bikesharing_constants

NUMERIC_FEATURE_KEYS = bikesharing_constants.NUMERIC_FEATURE_KEYS
CATEGORICAL_FEATURE_KEYS = bikesharing_constants.CATEGORICAL_FEATURE_KEYS
KEY_COLUMN = bikesharing_constants.KEY_COLUMN
LABEL_COLUMN = bikesharing_constants.LABEL_COLUMN

transformed_name = bikesharing_constants.transformed_name

# A function to scale numerical features and label encode categorical features
def preprocessing_fn(inputs):
      
    outputs = {}
    
    for key in NUMERIC_FEATURE_KEYS:
        outputs[transformed_name(key)] = tft.scale_to_z_score(_fill_in_missing(inputs[key]))
    
    for key in CATEGORICAL_FEATURE_KEYS:    
        outputs[transformed_name(key)] = _fill_in_missing(inputs[key])
        tft.vocabulary(inputs[key], vocab_filename=key)    

    outputs[transformed_name(LABEL_COLUMN)] = _fill_in_missing(inputs[LABEL_COLUMN])
    outputs[transformed_name(KEY_COLUMN)] = _fill_in_missing(inputs[KEY_COLUMN])
    
    return outputs

def _fill_in_missing(x):
    """Replace missing values in a SparseTensor.
    Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
    Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
    Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
    """
    default_value = '' if x.dtype == tf.string else 0
    return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)

Overwriting bikesharing_transform.py
This cell will be skipped during export to pipeline.


Now, we pass in this feature engineering code to the `Transform` component and run it to transform your data.

In [22]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_bikesharing_transform_module_file))
context.run(transform)

INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
Instructions for updating:
Use ref() instead.


INFO:absl:Using 1 process(es) for Beam pipeline execution.


  name: "datetime"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "daytype"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "daytype"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "season"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "season"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "weather"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "weather"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "atemp"
  value_count {
    min: 1
    max: 1
  }
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "casual"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "count"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fra

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transform_graph/13/.temp_path/tftransform_tmp/a1efa1dc762543289aa4d888350c675b/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note thi

...

  name: "atemp"
  type: FLOAT
}
feature {
  name: "count"
  type: INT
}
feature {
  name: "daytype"
  type: BYTES
}
feature {
  name: "humidity"
  type: INT
}
feature {
  name: "season"
  type: BYTES
}
feature {
  name: "temp"
  type: FLOAT
}
feature {
  name: "weather"
  type: BYTES
}
feature {
  name: "windspeed"
  type: FLOAT
}
}), (<PCollection[Analyze/CreateSavedModel/BindTensors/ReplaceWithConstants.None] at 0x7f3704790390>, BeamDatasetMetadata(dataset_metadata={'_schema': feature {
  name: "atemp"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "count"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "daytype"
  type: BYTES
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "humidity"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "season"
  type: BYTES
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "temp"
  type: FLOA

  name: "atemp"
  type: FLOAT
}
feature {
  name: "count"
  type: INT
}
feature {
  name: "daytype"
  type: BYTES
}
feature {
  name: "humidity"
  type: INT
}
feature {
  name: "season"
  type: BYTES
}
feature {
  name: "temp"
  type: FLOAT
}
feature {
  name: "weather"
  type: BYTES
}
feature {
  name: "windspeed"
  type: FLOAT
}
}), (<PCollection[Analyze/CreateSavedModel/BindTensors/ReplaceWithConstants.None] at 0x7f3704790390>, BeamDatasetMetadata(dataset_metadata={'_schema': feature {
  name: "atemp"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "count"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "daytype"
  type: BYTES
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "humidity"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "season"
  type: BYTES
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "temp"
  type: FLOA

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:Assets written to: gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transform_graph/13/.temp_path/tftransform_tmp/1af2427b53314652a63c9657afee3710/assets
INFO:tensorflow:SavedModel written to: gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transform_graph/13/.temp_path/tftransform_tmp/1af2427b53314652a63c9657afee3710/saved_model.pb
value: "\n\013\n\tConst_3:0\022\006season"

value: "\n\013\n\tConst_5:0\022\007weather"

value: "\n\013\n\tConst_7:0\022\007daytype"

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
value: "\n\013\n\tConst_3:0\022\006season"

value: "\n\013\n\tConst_5:0\022\007weather"

value: "\n\013\n\tConst_7:0\022\007daytype"

INFO:tensorflow:Saver not created because there are no

INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 13
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transform_graph/13, id: 14)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transformed_examples/13, id: 15)]
        ))

Let's examine the output artifacts of `Transform`. This component produces two types of outputs:

* `transform_graph` is the graph that can perform the preprocessing operations (this graph will be included in the serving and evaluation models).
* `transformed_examples` represents the preprocessed training and evaluation data.

In [23]:
%%skip_for_export

transform.outputs

{'transform_graph': Channel(
    type_name: TransformGraph
    artifacts: [Artifact(type_name: TransformGraph, uri: gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transform_graph/13, id: 14)]
), 'transformed_examples': Channel(
    type_name: Examples
    artifacts: [Artifact(type_name: Examples, uri: gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transformed_examples/13, id: 15)]
)}

This cell will be skipped during export to pipeline.


Take a peek at the `transform_graph` artifact.  It points to a directory containing three subdirectories.

In [24]:
%%skip_for_export

train_uri = transform.outputs['transform_graph'].get()[0].uri

# Get the list of files in this directory
transform_graph_filenames = []
for blob in client.list_blobs(_bucket, prefix=train_uri.replace("gs://" + _bucket + "/", "")):
    transform_graph_filenames.append("gs://" + _bucket + "/" + str(blob.name))
transform_graph_filenames = transform_graph_filenames[1:]

pp.pprint(transform_graph_filenames)

['gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transform_graph/13/metadata/',
 'gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transform_graph/13/metadata/schema.pbtxt',
 'gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transform_graph/13/transform_fn/',
 'gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transform_graph/13/transform_fn/assets/',
 'gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transform_graph/13/transform_fn/assets/daytype',
 'gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transform_graph/13/transform_fn/assets/season',
 'gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transform_graph/13/transform_fn/assets/weather',
 'gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transform_graph/13/transform_fn/saved_model.pb',
 'gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transform_graph/13/transform_fn/variables/',
 'gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform

The `transformed_metadata` subdirectory contains the schema of the preprocessed data. The `transform_fn` subdirectory contains the actual preprocessing graph. The `metadata` subdirectory contains the schema of the original data.

We can also take a look at the first three transformed examples:

In [25]:
%%skip_for_export

# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'train')

# Get the list of files in this directory
tfrecord_filenames = []
for blob in client.list_blobs(_bucket, prefix=train_uri.replace("gs://" + _bucket + "/", "")):
    tfrecord_filenames.append("gs://" + _bucket + "/" + str(blob.name))
tfrecord_filenames = tfrecord_filenames[1:]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "atemp"
    value {
      float_list {
        value: -1.1852953433990479
      }
    }
  }
  feature {
    key: "count"
    value {
      int64_list {
        value: 40
      }
    }
  }
  feature {
    key: "daytype"
    value {
      bytes_list {
        value: "weekend"
      }
    }
  }
  feature {
    key: "humidity"
    value {
      float_list {
        value: 0.9437353014945984
      }
    }
  }
  feature {
    key: "season"
    value {
      bytes_list {
        value: "spring"
      }
    }
  }
  feature {
    key: "temp"
    value {
      float_list {
        value: -1.4365949630737305
      }
    }
  }
  feature {
    key: "weather"
    value {
      bytes_list {
        value: "clear"
      }
    }
  }
  feature {
    key: "windspeed"
    value {
      float_list {
        value: -1.57523775100708
      }
    }
  }
}

features {
  feature {
    key: "atemp"
    value {
      float_list {
        value: -1.0954102277755737
      }
    }
  }


After the `Transform` component has transformed your data into features, and the next step is to train a model.

### Trainer
The `Trainer` component will train a model that you define in TensorFlow. Default Trainer support Estimator API, to use Keras API, you need to specify [Generic Trainer](https://github.com/tensorflow/community/blob/master/rfcs/20200117-tfx-generic-trainer.md) by setup `custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor)` in Trainer's contructor.

`Trainer` takes as input the schema from `SchemaGen`, the transformed data and graph from `Transform`, training parameters, as well as a module that contains user-defined model code.

Let's see an example of user-defined model code below (for an introduction to the TensorFlow Keras APIs, [see the tutorial](https://www.tensorflow.org/guide/keras)):

In [26]:
_bikesharing_trainer_module_file = 'bikesharing_trainer.py'

In [27]:
%%skip_for_export
%%writefile {_bikesharing_trainer_module_file}

from typing import List, Text

import os
import absl
import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx.components.trainer.executor import TrainerFnArgs

import bikesharing_constants

KEY_COLUMN = bikesharing_constants.KEY_COLUMN
LABEL_COLUMN = bikesharing_constants.LABEL_COLUMN
NUMERIC_FEATURE_KEYS = bikesharing_constants.NUMERIC_FEATURE_KEYS
CATEGORICAL_FEATURE_KEYS = bikesharing_constants.CATEGORICAL_FEATURE_KEYS

HIDDEN_UNITS_1 = bikesharing_constants.HIDDEN_UNITS_1
HIDDEN_UNITS_2 = bikesharing_constants.HIDDEN_UNITS_2
HIDDEN_UNITS_3 = bikesharing_constants.HIDDEN_UNITS_3
BATCH_SIZE = bikesharing_constants.BATCH_SIZE
NUM_EPOCHS = bikesharing_constants.NUM_EPOCHS
LEARNING_RATE = bikesharing_constants.LEARNING_RATE

transformed_name = bikesharing_constants.transformed_name


def _gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed files."""
    return tf.data.TFRecordDataset(
      filenames,
      compression_type='GZIP')

def get_dataset_size(file_path):
    """Function that fetchs the size of the Tfrecords dataset."""
    size = 1
    file_list = tf.io.gfile.glob(file_path)
    for file in file_list:
        for record in tf.compat.v1.io.tf_record_iterator(file, options=tf.io.TFRecordOptions(
    compression_type='GZIP')):
            size += 1
    return size

def _get_serve_tf_examples_fn(model, label_column, tf_transform_output):
    """Returns a function that parses a serialized tf.Example and applies TFT."""

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(label_column)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        return model(transformed_features)

    return serve_tf_examples_fn

def _input_fn(file_pattern: List[Text],
              label_column,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 16) -> tf.data.Dataset:
    """Generates features and label for tuning/training.

    Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

    Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
    """
    transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())

    dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transformed_feature_spec,
      reader=_gzip_reader_fn,
      label_key=transformed_name(label_column))

    return dataset



def _build_keras_model(tf_transform_output: tft.TFTransformOutput,
                       hidden_units: List[int] = None,
                       learning_rate: float = 0.01) -> tf.keras.Model:
    """Creates a DNN Keras model for classifying taxi data.

    Args:
    hidden_units: [int], the layer sizes of the DNN (input layer first).

    Returns:
    A keras Model.
    """
    numeric_columns = [
      tf.feature_column.numeric_column(transformed_name(key), shape=())
      for key in NUMERIC_FEATURE_KEYS
    ]
    
    categorical_columns = [
      tf.feature_column.categorical_column_with_vocabulary_file(
        transformed_name(key), 
        vocabulary_file=tf_transform_output.vocabulary_file_by_name(
            vocab_filename=key), 
        dtype=tf.dtypes.string,
        default_value=None, 
        num_oov_buckets=0)
      for key in CATEGORICAL_FEATURE_KEYS
    ]
    
    indicator_columns = [
      tf.feature_column.indicator_column(categorical_column)
      for categorical_column in categorical_columns
    ]
        
        
    model = dnn_regressor(
      input_columns=numeric_columns + indicator_columns,
      dnn_hidden_units=hidden_units or [16, 16, 16],
      learning_rate=learning_rate)
    return model


def dnn_regressor(input_columns, dnn_hidden_units, learning_rate):
    """Build a simple keras wide and deep model.

    Args:
    wide_columns: Feature columns wrapped in indicator_column for wide (linear)
      part of the model.
    deep_columns: Feature columns for deep part of the model.
    dnn_hidden_units: [int], the layer sizes of the hidden DNN.

    Returns:
    A Wide and Deep Keras model
    """
    # Following values are hard coded for simplicity in this example,
    # However prefarably they should be passsed in as hparams.

    input_layers = {
      colname: tf.keras.layers.Input(name=transformed_name(colname), shape=(), dtype=tf.float32)
      for colname in NUMERIC_FEATURE_KEYS
    }
    input_layers.update({
      colname: tf.keras.layers.Input(name=transformed_name(colname), shape=(), dtype=tf.string)
      for colname in CATEGORICAL_FEATURE_KEYS
    })

    deep = tf.keras.layers.DenseFeatures(input_columns)(input_layers)
    for numnodes in dnn_hidden_units:
        deep = tf.keras.layers.Dense(numnodes, activation='relu')(deep)

    output = tf.keras.layers.Dense(
      1, activation=None)(deep)

    model = tf.keras.Model(input_layers, output)
    model.compile(
        optimizer = tf.keras.optimizers.Adam(lr=learning_rate),
        loss = "mean_squared_error",
        metrics=[tf.keras.metrics.MeanSquaredError()])
    model.summary(print_fn=absl.logging.info)
    return model


# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
    """Train the model based on given args.

    Args:
    fn_args: Holds args used to train the model as name/value pairs.
    """

    # Training set size
    TRAIN_SIZE = get_dataset_size(fn_args.train_files)
    NUM_STEPS = TRAIN_SIZE / BATCH_SIZE # number of steps per epoch for which to train model
    
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    
    train_dataset = _input_fn(fn_args.train_files, LABEL_COLUMN, tf_transform_output, BATCH_SIZE)
    eval_dataset = _input_fn(fn_args.eval_files, LABEL_COLUMN, tf_transform_output, BATCH_SIZE)

    model = _build_keras_model(
        tf_transform_output,
        hidden_units=[HIDDEN_UNITS_1, HIDDEN_UNITS_2, HIDDEN_UNITS_3],
        learning_rate=LEARNING_RATE)

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir, update_freq='batch')
    
    model.fit(
      train_dataset,
      epochs=NUM_EPOCHS,        
      steps_per_epoch=NUM_STEPS,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

    signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(model,
                                    LABEL_COLUMN,
                                    tf_transform_output).get_concrete_function(
                                        tf.TensorSpec(
                                            shape=[None],
                                            dtype=tf.string,
                                            name='examples')),
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting bikesharing_trainer.py
This cell will be skipped during export to pipeline.


Now, we pass in this model code to the `Trainer` component and run it to train the model.

In [28]:
trainer = Trainer(
    module_file=os.path.abspath(_bikesharing_trainer_module_file),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(),
    eval_args=trainer_pb2.EvalArgs(num_steps=50))
context.run(trainer)

INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:absl:Training model.


INFO:tensorflow:vocabulary_size = 4 in season is inferred from the number of elements in the vocabulary_file gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transform_graph/13/transform_fn/assets/season.
INFO:tensorflow:vocabulary_size = 4 in weather is inferred from the number of elements in the vocabulary_file gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transform_graph/13/transform_fn/assets/weather.
INFO:tensorflow:vocabulary_size = 3 in daytype is inferred from the number of elements in the vocabulary_file gs://bike-sharing-pipeline-metadata/tfx-pipeline/Transform/transform_graph/13/transform_fn/assets/daytype.


INFO:absl:Model: "model"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:atemp (InputLayer)              [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:daytype (InputLayer)            [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:humidity (InputLayer)           [(None,)]            0                                            
INFO:absl:_____________________________________________________________________________________________

58/57 [==============================] - 26s 456ms/step - loss: 57809.5859 - mean_squared_error: 57809.5859 - val_loss: 27045.7441 - val_mean_squared_error: 27045.7441
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: gs://bike-sharing-pipeline-metadata/tfx-pipeline/Trainer/model/14/serving_model_dir/assets


INFO:absl:Training complete. Model written to gs://bike-sharing-pipeline-metadata/tfx-pipeline/Trainer/model/14/serving_model_dir
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 14
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: gs://bike-sharing-pipeline-metadata/tfx-pipeline/Trainer/model/14, id: 16)]
        ))

#### Analyze Training with TensorBoard
Take a peek at the trainer artifact. It points to a directory containing the model subdirectories.

In [29]:
%%skip_for_export

model_artifact_dir = trainer.outputs['model'].get()[0].uri

# Get the list of files in this directory
model_artifact_filenames = []
for blob in client.list_blobs(_bucket, prefix=model_artifact_dir.replace("gs://" + _bucket + "/", "")):
    model_artifact_filenames.append("gs://" + _bucket + "/" + str(blob.name))
model_artifact_filenames = model_artifact_filenames[1:]

pp.pprint(model_artifact_filenames)

model_dir = os.path.join(model_artifact_dir, 'serving_model_dir')

# Get the list of files in this directory
model_dir_filenames = []
for blob in client.list_blobs(_bucket, prefix=model_dir.replace("gs://" + _bucket + "/", "")):
    model_dir_filenames.append("gs://" + _bucket + "/" + str(blob.name))
model_dir_filenames = model_dir_filenames[1:]

pp.pprint(model_dir_filenames)

['gs://bike-sharing-pipeline-metadata/tfx-pipeline/Trainer/model/14/logs/',
 'gs://bike-sharing-pipeline-metadata/tfx-pipeline/Trainer/model/14/logs/train/',
 'gs://bike-sharing-pipeline-metadata/tfx-pipeline/Trainer/model/14/logs/train/events.out.tfevents.1599568507.bike-sharing-tensorflow-2-3.26008.2859.v2',
 'gs://bike-sharing-pipeline-metadata/tfx-pipeline/Trainer/model/14/logs/train/events.out.tfevents.1599568509.bike-sharing-tensorflow-2-3.profile-empty',
 'gs://bike-sharing-pipeline-metadata/tfx-pipeline/Trainer/model/14/logs/train/plugins/',
 'gs://bike-sharing-pipeline-metadata/tfx-pipeline/Trainer/model/14/logs/train/plugins/profile/',
 'gs://bike-sharing-pipeline-metadata/tfx-pipeline/Trainer/model/14/logs/train/plugins/profile/2020_09_08_12_35_09/',
 'gs://bike-sharing-pipeline-metadata/tfx-pipeline/Trainer/model/14/logs/train/plugins/profile/2020_09_08_12_35_09/bike-sharing-tensorflow-2-3.input_pipeline.pb',
 'gs://bike-sharing-pipeline-metadata/tfx-pipeline/Trainer/model/

Optionally, we can connect TensorBoard to the Trainer to analyze our model's training curves.

In [30]:
%%skip_for_export

log_dir = os.path.join(model_artifact_dir, 'logs')

%load_ext tensorboard
%tensorboard --logdir {log_dir}

This cell will be skipped during export to pipeline.


### Evaluator
The `Evaluator` component computes model performance metrics over the evaluation set. It uses the [TensorFlow Model Analysis](https://www.tensorflow.org/tfx/model_analysis/get_started) library. The `Evaluator` can also optionally validate that a newly trained model is better than the previous model. This is useful in a production pipeline setting where you may automatically train and validate a model every day. In this notebook, we only train one model, so the `Evaluator` automatically will label the model as "good". 

`Evaluator` will take as input the data from `ExampleGen`, the trained model from `Trainer`, and slicing configuration. The slicing configuration allows you to slice your metrics on feature values (e.g. how does your model perform on taxi trips that start at 8am versus 8pm?). See an example of this configuration below:

In [31]:
import bikesharing_constants

KEY_COLUMN = bikesharing_constants.KEY_COLUMN
LABEL_COLUMN = bikesharing_constants.LABEL_COLUMN
NUMERIC_FEATURE_KEYS = bikesharing_constants.NUMERIC_FEATURE_KEYS
CATEGORICAL_FEATURE_KEYS = bikesharing_constants.CATEGORICAL_FEATURE_KEYS

transformed_name = bikesharing_constants.transformed_name

eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and 
        # remove the label_key.
        tfma.ModelSpec(label_key=LABEL_COLUMN)
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='MeanSquaredError',
                      threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          upper_bound={'value': 32000}),
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.LOWER_IS_BETTER,
                          absolute={'value':1000})))
            ]
        )
    ],
    slicing_specs=
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        [tfma.SlicingSpec()] + 
        # Data can be sliced along any feature column.
        [tfma.SlicingSpec(feature_keys=[transformed_name(feature_key)]) for feature_key in CATEGORICAL_FEATURE_KEYS]
)

Next, we give this configuration to `Evaluator` and run it.

In [32]:
# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

# The model resolver is only required if performing model validation in addition
# to evaluation. In this case we validate against the latest blessed model. If
# no model has been blessed before (as in this case) the evaluator will make our
# candidate the first blessed model.
model_resolver = ResolverNode(
      instance_name='latest_blessed_model_resolver',
      resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing))
context.run(model_resolver)

evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    # Change threshold will be ignored if there is no baseline (first run).
    eval_config=eval_config)
context.run(evaluator)

INFO:absl:Running driver for ResolverNode.latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for ResolverNode.latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is likely because a baseline model was not provided: updated_config=
model_specs {
  label_key: "count"
}
slicing_specs {
}
slicing_specs {
  feature_keys: "season"
}
slicing_specs {
  feature_keys: "weather"
}
slicing_specs {
  feature_keys: "daytype"
}
metrics_specs {
  metrics {
    class_name: "ExampleCount"
    threshold {
    }
  }
  metrics {
    class_name: "MeanSquaredError"
    threshold {
      value_threshold {
        upper_bound {
          value: 32000.0
        }
      }
    }
  }
}

INFO:absl:U

ExecutionResult(
    component_id: Evaluator
    execution_id: 16
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(type_name: ModelEvaluation, uri: gs://bike-sharing-pipeline-metadata/tfx-pipeline/Evaluator/evaluation/16, id: 17)]
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(type_name: ModelBlessing, uri: gs://bike-sharing-pipeline-metadata/tfx-pipeline/Evaluator/blessing/16, id: 18)]
        ))

Now let's examine the output artifacts of `Evaluator`. 

In [33]:
%%skip_for_export

evaluator.outputs

{'evaluation': Channel(
    type_name: ModelEvaluation
    artifacts: [Artifact(type_name: ModelEvaluation, uri: gs://bike-sharing-pipeline-metadata/tfx-pipeline/Evaluator/evaluation/16, id: 17)]
), 'blessing': Channel(
    type_name: ModelBlessing
    artifacts: [Artifact(type_name: ModelBlessing, uri: gs://bike-sharing-pipeline-metadata/tfx-pipeline/Evaluator/blessing/16, id: 18)]
)}

This cell will be skipped during export to pipeline.


Using the `evaluation` output we can show the default visualization of global metrics on the entire evaluation set.

In [34]:
%%skip_for_export

context.show(evaluator.outputs['evaluation'])

This cell will be skipped during export to pipeline.


To see the visualization for sliced evaluation metrics, we can directly call the TensorFlow Model Analysis library.

In [35]:
%%skip_for_export

import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# Show data sliced along feature column trip_start_hour.
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column="season")

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'season:spring', 'met…

This cell will be skipped during export to pipeline.


This visualization shows the same metrics, but computed at every feature value of `season` instead of on the entire evaluation set.

TensorFlow Model Analysis supports many other visualizations, such as Fairness Indicators and plotting a time series of model performance. To learn more, see [the tutorial](https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic).

Since we added thresholds to our config, validation output is also available. The precence of a `blessing` artifact indicates that our model passed validation. Since this is the first validation being performed the candidate is automatically blessed.

In [37]:
%%skip_for_export

blessing_uri = evaluator.outputs.blessing.get()[0].uri
!gsutil ls -l {blessing_uri}

         0  2020-09-08T12:36:20Z  gs://bike-sharing-pipeline-metadata/tfx-pipeline/Evaluator/blessing/16/
         0  2020-09-08T12:37:31Z  gs://bike-sharing-pipeline-metadata/tfx-pipeline/Evaluator/blessing/16/BLESSED
TOTAL: 2 objects, 0 bytes (0 B)
This cell will be skipped during export to pipeline.


Now can also verify the success by loading the validation result record:

In [38]:
%%skip_for_export

PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

validation_ok: true

This cell will be skipped during export to pipeline.


### Pusher
The `Pusher` component is usually at the end of a TFX pipeline. It checks whether a model has passed validation, and if so, exports the model to `_serving_model_dir`.

In [39]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Pusher
INFO:absl:Model version: 1599568676
INFO:absl:Model written to serving path /tmp/tmpszgxiak6/serving_model/bikesharing/1599568676.
INFO:absl:Model pushed to gs://bike-sharing-pipeline-metadata/tfx-pipeline/Pusher/pushed_model/17.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 17
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(type_name: PushedModel, uri: gs://bike-sharing-pipeline-metadata/tfx-pipeline/Pusher/pushed_model/17, id: 19)]
        ))

Let's examine the output artifacts of `Pusher`. 

In [40]:
%%skip_for_export

pusher.outputs

{'pushed_model': Channel(
    type_name: PushedModel
    artifacts: [Artifact(type_name: PushedModel, uri: gs://bike-sharing-pipeline-metadata/tfx-pipeline/Pusher/pushed_model/17, id: 19)]
)}

This cell will be skipped during export to pipeline.


In particular, the Pusher will export your model in the SavedModel format, which looks like this:

In [41]:
%%skip_for_export

push_uri = pusher.outputs.model_push.get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)

('serving_default',
 <tensorflow.python.saved_model.load._WrapperFunction object at 0x7f369a5be490>)
This cell will be skipped during export to pipeline.


We're finished our tour of built-in TFX components!

After you're happy with experimenting with TFX components and code in this notebook, you may want to export it as a pipeline to be orchestrated with Apache Airflow or Apache Beam. See the final section.

## Export to pipeline

To export the contents of this notebook as a pipeline to be orchestrated with Airflow or Beam, follow the instructions below.

### 1. Select an orchestrator

In [42]:
_runner_type = 'airflow' #@param ["beam", "airflow"]
_pipeline_name = 'bikesharing_pipeline_%s' % _runner_type

### 3. Set up paths for the pipeline

In [45]:
#docs_infra: no_execute

_notebook_filepath = os.path.join(os.getcwd(), 'components_keras_gs_cloud_sql.ipynb')

_base_dir = os.path.join('gs://', _bucket)
_data_root = 'gs://bike-sharing-data/'

_tfx_root = os.path.join(_base_dir, 'tfx_pipeline_airflow')
_serving_model_dir = os.path.join(_tfx_root, 'serving_model')
_pipeline_root = os.path.join(_tfx_root, 'pipelines', _pipeline_name)
_metadata_path = os.path.join(_tfx_root, 'metadata', _pipeline_name,
                              'metadata.db')

### 4. Choose components to include in the pipeline

In [46]:
#docs_infra: no_execute
components = [
    example_gen, statistics_gen, schema_gen, example_validator, transform,
    trainer, evaluator, pusher
]

### 5. Generate pipeline files

In [47]:
%%skip_for_export
#docs_infra: no_execute

#@markdown Run this cell to generate the pipeline files.

if get_ipython().magics_manager.auto_magic:
  print('Warning: %automagic is ON. Line magics specified without the % prefix '
        'will not be scrubbed during export to pipeline.')

_pipeline_export_filepath = '%s.py' % _pipeline_name
context.export_to_pipeline(notebook_filepath=_notebook_filepath,
                           export_filepath=_pipeline_export_filepath,
                           runner_type=_runner_type)

INFO:absl:Exporting contents of /home/jupyter/e2e-ml-pipeline/tfx-components/tutorial/components_keras_gs_cloud_sql/components_keras_gs_cloud_sql.ipynb to bikesharing_pipeline_airflow.py with airflow runner.
INFO:absl:21 cell(s) marked with "%%skip_for_export", skipped.


This cell will be skipped during export to pipeline.
